In [32]:
from sklearn.ensemble import RandomForestClassifier
import os
os.chdir('/Users/ritwikdeshpande/DLH/Project/CS598_DLH_Project/')
import weka.core.jvm as jvm
from weka.core.converters import Loader
from weka.filters import Filter
from weka.attribute_selection import ASEvaluation, AttributeSelection
from weka.classifiers import Classifier, Evaluation
from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import KFold
import csv
import pandas as pd
import collections

jvm.start()

INFO:weka.core.jvm:JVM already running, call jvm.stop() first


In [55]:
class RandomForest:
    def __init__(self, x_train, y_train, x_test, y_test, k):
        self.rf = RandomForestClassifier(n_estimators=100, random_state=42)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.k = k
        print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def train(self):
        self.rf.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.rf.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro
    
    def feature_selection_InfoGainAttributeEval(self, morbidity):
        loader = Loader(classname="weka.core.converters.ArffLoader")
        train_data = loader.load_file(f"./dataset/train/train_{morbidity}_tfidf.arff")
        train_data.class_is_last()

        # Initialize attribute selection
        eval = ASEvaluation(classname="weka.attributeSelection.InfoGainAttributeEval")
        search = AttributeSelection()
        search.evaluator = eval
        search.select_attributes(train_data)
        selected_attributes = search.selected_attributes
        filtered_attributes = np.delete(selected_attributes, [-1])
        print("Selected attributes:", type(filtered_attributes), filtered_attributes.shape)

        # Apply selected attributes to the training and testing sets
        self.x_train = self.x_train[:, filtered_attributes]
        self.x_test = self.x_test[:, filtered_attributes]

In [43]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

In [56]:
column_headings = ["Morbidity Class", "RF_Macro F1", "RF_Micro F1"]

with open("./results/tf-idf-features/performance_RF.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            rf_obj = RandomForest(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5)
            rf_obj.feature_selection_InfoGainAttributeEval(morbidity)
            rf_obj.train()

            f1_macro, f1_micro = rf_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
        
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_RF.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_RF.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
(514, 600) (514,) (58, 600) (58,)
Selected attributes: <class 'numpy.ndarray'> (5,)
(514, 600) (514,) (58, 600) (58,)
Selected attributes: <class 'numpy.ndarray'> (5,)
(515, 600) (515,) (57, 600) (57,)
Selected attributes: <class 'numpy.ndarray'> (5,)
(515, 600) (515,) (57, 600) (57,)
Selected attributes: <class 'numpy.ndarray'> (5,)
(515, 600) (515,) (57, 600) (57,)
Selected attributes: <class 'numpy.ndarray'> (5,)
(515, 600) (515,) (57, 600) (57,)
Selected attributes: <class 'numpy.ndarray'> (5,)
(515, 600) (515,) (57, 600) (57,)
Selected attributes: <class 'numpy.ndarray'> (5,)
(515, 600) (515,) (57, 600) (57,)
Selected attributes: <class 'numpy.ndarray'> (5,)
(515, 600) (515,) (57, 600) (57,)
Selected attributes: <class 'numpy.ndarray'> (5,)
(515, 600) (515,) (57, 600) (57,)
Selected attributes: <class 'numpy.ndarray'> (5,)
Macro F1 score: 0.9556492779857066 and Micro F1 Score 0.9824561403508772
CAD
(548, 600) (548,) Counter({1.